In [1]:
import sys
sys.dont_write_bytecode = True
import numpy as np
from testing_suite import *
from parameters import *
import functions as fcn

In [2]:
theory_parameters = Parameters(code='T', beta_functions=None, fixed_point=None, true_result=0.15, linear_generating_function=None)
meta_parameters = Parameters(code='M', collocation_point_numbers=None, collocation_point_bounds=None, basis_functions=None, basis_functions_derivatives=None, fix_parameters=None, population_size=None, algorithm=None)
control_parameters = Parameters(code='C', fitness_function=None, selection=None, mutation=None, crossover=None, iteration_cap=5000)

In [3]:
def add(a,b):
    return a+b

def mul(a,b):
    return a*b

test_parameters = Parameters(code='Z', small=np.arange(1,11), medium=np.arange(0,120,20), large=np.arange(0,1250,250), result=0.15, functions=np.array([add, mul]))
print(test_parameters.get_option_counts())
print(test_parameters.get_combination_number())


{'small': 10, 'functions': 2, 'medium': 6, 'result': 1, 'large': 5}
600
